In [1]:
import logging

import numpy

from cicada.communicator import SocketCommunicator
from cicada.logging import Logger
from cicada.shamir import ShamirBasicProtocolSuite

logging.basicConfig(level=logging.INFO)

def main(communicator):
    # One-time initialization.
    log = Logger(logging.getLogger(), communicator=communicator)
    protocol = ShamirBasicProtocolSuite(communicator=communicator, threshold=2)
    
    total_share = protocol.share(src=0, secret=numpy.array(0), shape=())

    # Main iteration loop.
    for iteration in range(0, 8):
        # Increment the total.
        contributor = iteration % communicator.world_size
        increment = numpy.array(1) if communicator.rank == contributor else None
        increment_share = protocol.share(src=contributor, secret=increment, shape=())
        total_share = protocol.add(total_share, increment_share)
    
        # Print the current total.
        total = protocol.reveal(total_share)
        log.info(f"Iteration {iteration} comm {communicator.name} total: {total}", src=0)

SocketCommunicator.run(world_size=4, fn=main);

INFO:root:Iteration 0 comm world total: 1.0
INFO:root:Iteration 1 comm world total: 2.0
INFO:root:Iteration 2 comm world total: 3.0
INFO:root:Iteration 3 comm world total: 4.0
INFO:root:Iteration 4 comm world total: 5.0
INFO:root:Iteration 5 comm world total: 6.0
INFO:root:Iteration 6 comm world total: 7.0
INFO:root:Iteration 7 comm world total: 8.0


In [2]:
import os
import signal

def main(communicator):
    # One-time initialization.
    log = Logger(logging.getLogger(), communicator=communicator)
    protocol = ShamirBasicProtocolSuite(communicator=communicator, threshold=2)
    
    total_share = protocol.share(src=0, secret=numpy.array(0), shape=())

    # Main iteration loop.
    for iteration in range(0, 8):
        # Simulate the unexpected death of player 3.
        if iteration == 4 and communicator.rank == 3:
            os.kill(os.getpid(), signal.SIGKILL)

        # Increment the total.
        contributor = iteration % communicator.world_size
        increment = numpy.array(1) if communicator.rank == contributor else None
        increment_share = protocol.share(src=contributor, secret=increment, shape=())
        total_share = protocol.add(total_share, increment_share)
    
        # Print the current total.
        total = protocol.reveal(total_share)
        log.info(f"Iteration {iteration} comm {communicator.name} total: {total}", src=0)

SocketCommunicator.run(world_size=4, fn=main);

INFO:root:Iteration 0 comm world total: 1.0
INFO:root:Iteration 1 comm world total: 2.0
INFO:root:Iteration 2 comm world total: 3.0
INFO:root:Iteration 3 comm world total: 4.0
ERROR:cicada.communicator.socket:Comm world player 0 failed: Timeout('Tag GATHERV from player 3 timed-out after 5s')
ERROR:cicada.communicator.socket:Comm world player 1 failed: Timeout('Tag GATHERV from player 0 timed-out after 5s')
ERROR:cicada.communicator.socket:Comm world player 2 failed: Timeout('Tag GATHERV from player 0 timed-out after 5s')
ERROR:cicada.communicator.socket:Comm world player 3 failed: Terminated(exitcode=-9)


In [3]:
def main(communicator):
    # One-time initialization.
    log = Logger(logging.getLogger(), communicator=communicator)
    protocol = ShamirBasicProtocolSuite(communicator=communicator, threshold=2)
    
    total_share = protocol.share(src=0, secret=numpy.array(0), shape=())

    # Main iteration loop.
    for iteration in range(0, 8):
        # Simulate the unexpected death of player 3.
        if iteration == 4 and communicator.rank == 3:
            os.kill(os.getpid(), signal.SIGKILL)

        # Do computation in this block.
        try:
            # Increment the total.
            contributor = iteration % communicator.world_size
            increment = numpy.array(1) if communicator.rank == contributor else None
            increment_share = protocol.share(src=contributor, secret=increment, shape=())
            total_share = protocol.add(total_share, increment_share)
    
            # Print the current total.
            total = protocol.reveal(total_share)
            log.info(f"Iteration {iteration} comm {communicator.name} total: {total}", src=0)

        # Implement failure recovery in this block.  Be careful here! Many
        # operations can't be used when there are unresponsive players.
        except Exception as e:
            log.sync = False
            log.error(f"Iteration {iteration} comm {communicator.name} player {communicator.rank} exception: {e}")
            break
    
SocketCommunicator.run(world_size=4, fn=main);

INFO:root:Iteration 0 comm world total: 1.0
INFO:root:Iteration 1 comm world total: 2.0
INFO:root:Iteration 2 comm world total: 3.0
INFO:root:Iteration 3 comm world total: 4.0
ERROR:root:Iteration 4 comm world player 0 exception: Tag GATHERV from player 3 timed-out after 5s
ERROR:root:Iteration 4 comm world player 2 exception: Tag GATHERV from player 0 timed-out after 5s
ERROR:root:Iteration 4 comm world player 1 exception: Tag GATHERV from player 0 timed-out after 5s
ERROR:cicada.communicator.socket:Comm world player 3 failed: Terminated(exitcode=-9)


In [4]:
def main(communicator):
    # One-time initialization.
    log = Logger(logging.getLogger(), communicator=communicator)
    protocol = ShamirBasicProtocolSuite(communicator=communicator, threshold=2)
    
    total_share = protocol.share(src=0, secret=numpy.array(0), shape=())

    # Main iteration loop.
    for iteration in range(0, 8):
        # Simulate the unexpected death of player 3.
        if iteration == 4 and communicator.rank == 3:
            os.kill(os.getpid(), signal.SIGKILL)

        # Do computation in this block.
        try:
            # Increment the total.
            contributor = iteration % communicator.world_size
            increment = numpy.array(1) if communicator.rank == contributor else None
            increment_share = protocol.share(src=contributor, secret=increment, shape=())
            total_share = protocol.add(total_share, increment_share)
    
            # Print the current total.
            total = protocol.reveal(total_share)
            log.info(f"Iteration {iteration} comm {communicator.name} total: {total}", src=0)

        # Implement failure recovery in this block.  Be careful here! Many
        # operations can't be used when there are unresponsive players.
        except Exception as e:
            log.sync = False
            log.error(f"Iteration {iteration} comm {communicator.name} player {communicator.rank} exception: {e}")

            # Something went wrong.  Revoke the current communicator to
            # ensure that all players are aware of it.
            communicator.revoke()

            # Obtain a new communicator that contains the remaining players.
            newcommunicator, oldranks = communicator.shrink(name="fallback")

            # Recreate the logger since objects that depend on the old,
            # revoked communicator must be rebuilt from scratch using the
            # new communicator.
            log = Logger(logging.getLogger(), newcommunicator)
            log.info(f"Iteration {iteration} shrank comm {communicator.name} with {communicator.world_size} players to comm {newcommunicator.name} with {newcommunicator.world_size} players.", src=0)

            # Recreate the protocol since objects that depend on the old,
            # revoked communicator must be rebuilt from scratch using the
            # new communicator.
            protocol = ShamirBasicProtocolSuite(newcommunicator, threshold=2, indices=protocol.indices[oldranks])

            # Cleanup the old communicator.
            communicator.free()
            communicator = newcommunicator
    
SocketCommunicator.run(world_size=4, fn=main);

INFO:root:Iteration 0 comm world total: 1.0
INFO:root:Iteration 1 comm world total: 2.0
INFO:root:Iteration 2 comm world total: 3.0
INFO:root:Iteration 3 comm world total: 4.0
ERROR:root:Iteration 4 comm world player 0 exception: Tag GATHERV from player 3 timed-out after 5s
ERROR:root:Iteration 4 comm world player 1 exception: Tag GATHERV from player 0 timed-out after 5s
ERROR:root:Iteration 4 comm world player 2 exception: Tag GATHERV from player 0 timed-out after 5s
INFO:root:Iteration 4 shrank comm world with 4 players to comm fallback with 3 players.
INFO:root:Iteration 5 comm fallback total: 6.0
INFO:root:Iteration 6 comm fallback total: 7.0
INFO:root:Iteration 7 comm fallback total: 8.0
ERROR:cicada.communicator.socket:Comm world player 3 failed: Terminated(exitcode=-9)


In [5]:
import os

def random_failure(pfail, seed):
    generator = numpy.random.default_rng(seed=seed)
    while True:
        if generator.uniform() <= pfail:
            os.kill(os.getpid(), signal.SIGKILL)
        yield

In [6]:
def main(communicator):
    # One-time initialization.
    failure = random_failure(pfail=0.05, seed=42 + communicator.rank)
    log = Logger(logging.getLogger(), communicator=communicator)
    protocol = ShamirBasicProtocolSuite(communicator=communicator, threshold=2)
    
    total_share = protocol.share(src=0, secret=numpy.array(0), shape=())

    # Main iteration loop.
    for iteration in range(0, 8):
        # Allow failures to occur at random.
        next(failure)

        # Do computation in this block.
        try:
            # Increment the total.
            contributor = iteration % communicator.world_size
            increment = numpy.array(1) if communicator.rank == contributor else None
            increment_share = protocol.share(src=contributor, secret=increment, shape=())
            total_share = protocol.add(total_share, increment_share)
    
            # Print the current total.
            total = protocol.reveal(total_share)
            log.info(f"Iteration {iteration} comm {communicator.name} total: {total}", src=0)

        # Implement failure recovery in this block.  Be careful here! Many
        # operations can't be used when there are unresponsive players.
        except Exception as e:
            log.sync = False
            log.error(f"Iteration {iteration} comm {communicator.name} player {communicator.rank} exception: {e}")

            # Something went wrong.  Revoke the current communicator to
            # ensure that all players are aware of it.
            communicator.revoke()

            # Obtain a new communicator that contains the remaining players.
            newcommunicator, oldranks = communicator.shrink(name="fallback")

            # Recreate the logger since objects that depend on the old,
            # revoked communicator must be rebuilt from scratch using the
            # new communicator.
            log = Logger(logging.getLogger(), newcommunicator)
            log.info(f"Iteration {iteration} shrank comm {communicator.name} with {communicator.world_size} players to comm {newcommunicator.name} with {newcommunicator.world_size} players.", src=0)

            # Recreate the protocol since objects that depend on the old,
            # revoked communicator must be rebuilt from scratch using the
            # new communicator.
            protocol = ShamirBasicProtocolSuite(newcommunicator, threshold=2, indices=protocol.indices[oldranks])

            # Cleanup the old communicator.
            communicator.free()
            communicator = newcommunicator
    
SocketCommunicator.run(world_size=4, fn=main);

INFO:root:Iteration 0 comm world total: 1.0
ERROR:root:Iteration 1 comm world player 0 exception: Tag SCATTER from player 1 timed-out after 5s
ERROR:root:Iteration 1 comm world player 3 exception: Tag SCATTER from player 1 timed-out after 5s
ERROR:root:Iteration 1 comm world player 2 exception: Tag SCATTER from player 1 timed-out after 5s
INFO:root:Iteration 1 shrank comm world with 4 players to comm fallback with 3 players.
INFO:root:Iteration 2 comm fallback total: 2.0
INFO:root:Iteration 3 comm fallback total: 3.0
INFO:root:Iteration 4 comm fallback total: 4.0
INFO:root:Iteration 5 comm fallback total: 5.0
INFO:root:Iteration 6 comm fallback total: 6.0
INFO:root:Iteration 7 comm fallback total: 7.0
ERROR:cicada.communicator.socket:Comm world player 1 failed: Terminated(exitcode=-9)


In [7]:
import itertools

def main(communicator):
    # One-time initialization.
    failure = random_failure(pfail=0.05, seed=42 + communicator.rank)
    log = Logger(logging.getLogger(), communicator=communicator)
    protocol = ShamirBasicProtocolSuite(communicator=communicator, threshold=2)
    
    total_share = protocol.share(src=0, secret=numpy.array(0), shape=())

    # Main iteration loop.
    for iteration in itertools.count():
        # Allow failures to occur at random.
        next(failure)

        # Do computation in this block.
        try:
            # Increment the total.
            contributor = iteration % communicator.world_size
            increment = numpy.array(1) if communicator.rank == contributor else None
            increment_share = protocol.share(src=contributor, secret=increment, shape=())
            total_share = protocol.add(total_share, increment_share)
    
            # Print the current total.
            total = protocol.reveal(total_share)
            log.info(f"Iteration {iteration} comm {communicator.name} total: {total}", src=0)

        # Implement failure recovery in this block.  Be careful here! Many
        # operations can't be used when there are unresponsive players.
        except Exception as e:
            log.sync = False
            log.error(f"Iteration {iteration} comm {communicator.name} player {communicator.rank} exception: {e}")

            # Something went wrong.  Revoke the current communicator to
            # ensure that all players are aware of it.
            communicator.revoke()

            # Obtain a new communicator that contains the remaining players.
            newcommunicator, oldranks = communicator.shrink(name="fallback")

            # Recreate the logger since objects that depend on the old,
            # revoked communicator must be rebuilt from scratch using the
            # new communicator.
            log = Logger(logging.getLogger(), newcommunicator)
            log.info(f"Iteration {iteration} shrank comm {communicator.name} with {communicator.world_size} players to comm {newcommunicator.name} with {newcommunicator.world_size} players.", src=0)

            # Recreate the protocol since objects that depend on the old,
            # revoked communicator must be rebuilt from scratch using the
            # new communicator.
            protocol = ShamirBasicProtocolSuite(newcommunicator, threshold=2, indices=protocol.indices[oldranks])

            # Cleanup the old communicator.
            communicator.free()
            communicator = newcommunicator
    
SocketCommunicator.run(world_size=4, fn=main);

INFO:root:Iteration 0 comm world total: 1.0
ERROR:root:Iteration 1 comm world player 0 exception: Tag SCATTER from player 1 timed-out after 5s
ERROR:root:Iteration 1 comm world player 2 exception: Tag SCATTER from player 1 timed-out after 5s
ERROR:root:Iteration 1 comm world player 3 exception: Tag SCATTER from player 1 timed-out after 5s
INFO:root:Iteration 1 shrank comm world with 4 players to comm fallback with 3 players.
INFO:root:Iteration 2 comm fallback total: 2.0
INFO:root:Iteration 3 comm fallback total: 3.0
INFO:root:Iteration 4 comm fallback total: 4.0
INFO:root:Iteration 5 comm fallback total: 5.0
INFO:root:Iteration 6 comm fallback total: 6.0
INFO:root:Iteration 7 comm fallback total: 7.0
INFO:root:Iteration 8 comm fallback total: 8.0
INFO:root:Iteration 9 comm fallback total: 9.0
INFO:root:Iteration 10 comm fallback total: 10.0
INFO:root:Iteration 11 comm fallback total: 11.0
INFO:root:Iteration 12 comm fallback total: 12.0
INFO:root:Iteration 13 comm fallback total: 13.0

In [8]:
def main(communicator):
    # One-time initialization.
    failure = random_failure(pfail=0.05, seed=42 + communicator.rank)
    log = Logger(logging.getLogger(), communicator=communicator)
    protocol = ShamirBasicProtocolSuite(communicator=communicator, threshold=2)
    
    total_share = protocol.share(src=0, secret=numpy.array(0), shape=())

    # Main iteration loop.
    for iteration in itertools.count():
        # Allow failures to occur at random.
        next(failure)

        # Do computation in this block.
        try:
            # Increment the total.
            contributor = iteration % communicator.world_size
            increment = numpy.array(1) if communicator.rank == contributor else None
            increment_share = protocol.share(src=contributor, secret=increment, shape=())
            total_share = protocol.add(total_share, increment_share)
    
            # Print the current total.
            total = protocol.reveal(total_share)
            log.info(f"Iteration {iteration} comm {communicator.name} total: {total}", src=0)

        # Implement failure recovery in this block.  Be careful here! Many
        # operations can't be used when there are unresponsive players.
        except Exception as e:
            log.sync = False
            log.error(f"Iteration {iteration} comm {communicator.name} player {communicator.rank} exception: {e}")

            # Something went wrong.  Revoke the current communicator to
            # ensure that all players are aware of it.
            communicator.revoke()

            # If we don't have enough players to continue, it's time to shutdown cleanly.
            if communicator.world_size == protocol.threshold:
                log.info(f"Iteration {iteration} not enough players to continue.", src=0)
                break

            # Obtain a new communicator that contains the remaining players.
            newcommunicator, oldranks = communicator.shrink(name="fallback")

            # Recreate the logger since objects that depend on the old,
            # revoked communicator must be rebuilt from scratch using the
            # new communicator.
            log = Logger(logging.getLogger(), newcommunicator)
            log.info(f"Iteration {iteration} shrank comm {communicator.name} with {communicator.world_size} players to comm {newcommunicator.name} with {newcommunicator.world_size} players.", src=0)

            # Recreate the protocol since objects that depend on the old,
            # revoked communicator must be rebuilt from scratch using the
            # new communicator.
            protocol = ShamirBasicProtocolSuite(newcommunicator, threshold=2, indices=protocol.indices[oldranks])

            # Cleanup the old communicator.
            communicator.free()
            communicator = newcommunicator
    
SocketCommunicator.run(world_size=4, fn=main);

INFO:root:Iteration 0 comm world total: 1.0
ERROR:root:Iteration 1 comm world player 0 exception: Tag SCATTER from player 1 timed-out after 5s
ERROR:root:Iteration 1 comm world player 3 exception: Tag SCATTER from player 1 timed-out after 5s
ERROR:root:Iteration 1 comm world player 2 exception: Tag SCATTER from player 1 timed-out after 5s
INFO:root:Iteration 1 shrank comm world with 4 players to comm fallback with 3 players.
INFO:root:Iteration 2 comm fallback total: 2.0
INFO:root:Iteration 3 comm fallback total: 3.0
INFO:root:Iteration 4 comm fallback total: 4.0
INFO:root:Iteration 5 comm fallback total: 5.0
INFO:root:Iteration 6 comm fallback total: 6.0
INFO:root:Iteration 7 comm fallback total: 7.0
INFO:root:Iteration 8 comm fallback total: 8.0
INFO:root:Iteration 9 comm fallback total: 9.0
INFO:root:Iteration 10 comm fallback total: 10.0
INFO:root:Iteration 11 comm fallback total: 11.0
INFO:root:Iteration 12 comm fallback total: 12.0
INFO:root:Iteration 13 comm fallback total: 13.0

In [9]:
def main(communicator):
    # One-time initialization.
    communicator_index = itertools.count(1)
    failure = random_failure(pfail=0.05, seed=42 + communicator.rank)
    log = Logger(logging.getLogger(), communicator=communicator)
    protocol = ShamirBasicProtocolSuite(communicator=communicator, threshold=2)
    
    total_share = protocol.share(src=0, secret=numpy.array(0), shape=())

    # Main iteration loop.
    for iteration in itertools.count():
        # Allow failures to occur at random.
        next(failure)

        # Do computation in this block.
        try:
            # Increment the total.
            contributor = iteration % communicator.world_size
            increment = numpy.array(1) if communicator.rank == contributor else None
            increment_share = protocol.share(src=contributor, secret=increment, shape=())
            total_share = protocol.add(total_share, increment_share)
    
            # Print the current total.
            total = protocol.reveal(total_share)
            log.info(f"Iteration {iteration} comm {communicator.name} total: {total}", src=0)

        # Implement failure recovery in this block.  Be careful here! Many
        # operations can't be used when there are unresponsive players.
        except Exception as e:
            log.sync = False
            log.error(f"Iteration {iteration} comm {communicator.name} player {communicator.rank} exception: {e}")

            # Something went wrong.  Revoke the current communicator to
            # ensure that all players are aware of it.
            communicator.revoke()

            # If we don't have enough players to continue, it's time to shutdown cleanly.
            if communicator.world_size == protocol.threshold:
                log.info(f"Iteration {iteration} not enough players to continue.", src=0)
                break

            # Obtain a new communicator that contains the remaining players.
            newcommunicator, oldranks = communicator.shrink(name=f"world-{next(communicator_index)}")

            # Recreate the logger since objects that depend on the old,
            # revoked communicator must be rebuilt from scratch using the
            # new communicator.
            log = Logger(logging.getLogger(), newcommunicator)
            log.info(f"Iteration {iteration} shrank comm {communicator.name} with {communicator.world_size} players to comm {newcommunicator.name} with {newcommunicator.world_size} players.", src=0)

            # Recreate the protocol since objects that depend on the old,
            # revoked communicator must be rebuilt from scratch using the
            # new communicator.
            protocol = ShamirBasicProtocolSuite(newcommunicator, threshold=2, indices=protocol.indices[oldranks])

            # Cleanup the old communicator.
            communicator.free()
            communicator = newcommunicator
    
SocketCommunicator.run(world_size=4, fn=main, name="world-0");

INFO:root:Iteration 0 comm world-0 total: 1.0
ERROR:root:Iteration 1 comm world-0 player 0 exception: Tag SCATTER from player 1 timed-out after 5s
ERROR:root:Iteration 1 comm world-0 player 3 exception: Tag SCATTER from player 1 timed-out after 5s
ERROR:root:Iteration 1 comm world-0 player 2 exception: Tag SCATTER from player 1 timed-out after 5s
INFO:root:Iteration 1 shrank comm world-0 with 4 players to comm world-1 with 3 players.
INFO:root:Iteration 2 comm world-1 total: 2.0
INFO:root:Iteration 3 comm world-1 total: 3.0
INFO:root:Iteration 4 comm world-1 total: 4.0
INFO:root:Iteration 5 comm world-1 total: 5.0
INFO:root:Iteration 6 comm world-1 total: 6.0
INFO:root:Iteration 7 comm world-1 total: 7.0
INFO:root:Iteration 8 comm world-1 total: 8.0
INFO:root:Iteration 9 comm world-1 total: 9.0
INFO:root:Iteration 10 comm world-1 total: 10.0
INFO:root:Iteration 11 comm world-1 total: 11.0
INFO:root:Iteration 12 comm world-1 total: 12.0
INFO:root:Iteration 13 comm world-1 total: 13.0
IN

In [10]:
logging.getLogger("cicada.communicator").setLevel(logging.CRITICAL)

def main(communicator):
    # One-time initialization.
    communicator_index = itertools.count(1)
    failure = random_failure(pfail=0.05, seed=42 + communicator.rank)
    log = Logger(logging.getLogger(), communicator=communicator)
    protocol = ShamirBasicProtocolSuite(communicator=communicator, threshold=2)
    
    total_share = protocol.share(src=0, secret=numpy.array(0), shape=())

    # Main iteration loop.
    for iteration in itertools.count():
        # Allow failures to occur at random.
        next(failure)

        # Do computation in this block.
        try:
            # Increment the total.
            contributor = iteration % communicator.world_size
            increment = numpy.array(1) if communicator.rank == contributor else None
            increment_share = protocol.share(src=contributor, secret=increment, shape=())
            total_share = protocol.add(total_share, increment_share)
    
            # Print the current total.
            total = protocol.reveal(total_share)
            log.info(f"Iteration {iteration} comm {communicator.name} total: {total}", src=0)

        # Implement failure recovery in this block.  Be careful here! Many
        # operations can't be used when there are unresponsive players.
        except Exception as e:
            # Something went wrong.  Revoke the current communicator to
            # ensure that all players are aware of it.
            communicator.revoke()

            # If we don't have enough players to continue, it's time to shutdown cleanly.
            if communicator.world_size == protocol.threshold:
                log.info(f"Iteration {iteration} not enough players to continue.", src=0)
                break

            # Obtain a new communicator that contains the remaining players.
            newcommunicator, oldranks = communicator.shrink(name=f"world-{next(communicator_index)}")

            # Recreate the logger since objects that depend on the old,
            # revoked communicator must be rebuilt from scratch using the
            # new communicator.
            log = Logger(logging.getLogger(), newcommunicator)
            log.info(f"Iteration {iteration} shrank comm {communicator.name} with {communicator.world_size} players to comm {newcommunicator.name} with {newcommunicator.world_size} players.", src=0)

            # Recreate the protocol since objects that depend on the old,
            # revoked communicator must be rebuilt from scratch using the
            # new communicator.
            protocol = ShamirBasicProtocolSuite(newcommunicator, threshold=2, indices=protocol.indices[oldranks])

            # Cleanup the old communicator.
            communicator.free()
            communicator = newcommunicator
    
SocketCommunicator.run(world_size=5, fn=main, name="world-0");

INFO:root:Iteration 0 comm world-0 total: 1.0
INFO:root:Iteration 1 shrank comm world-0 with 5 players to comm world-1 with 4 players.
INFO:root:Iteration 2 comm world-1 total: 2.0
INFO:root:Iteration 3 comm world-1 total: 3.0
INFO:root:Iteration 4 comm world-1 total: 4.0
INFO:root:Iteration 5 comm world-1 total: 5.0
INFO:root:Iteration 6 comm world-1 total: 6.0
INFO:root:Iteration 7 comm world-1 total: 7.0
INFO:root:Iteration 8 comm world-1 total: 8.0
INFO:root:Iteration 9 comm world-1 total: 9.0
INFO:root:Iteration 10 comm world-1 total: 10.0
INFO:root:Iteration 11 comm world-1 total: 11.0
INFO:root:Iteration 12 comm world-1 total: 12.0
INFO:root:Iteration 13 comm world-1 total: 13.0
INFO:root:Iteration 14 comm world-1 total: 14.0
INFO:root:Iteration 15 shrank comm world-1 with 4 players to comm world-2 with 3 players.
INFO:root:Iteration 16 comm world-2 total: 16.0
INFO:root:Iteration 17 comm world-2 total: 17.0
INFO:root:Iteration 18 comm world-2 total: 18.0
INFO:root:Iteration 19 